In [1]:
import os
import collections
import random
import json

from data_modules.domainnet_dataset import ImageDataset
from data_modules.domainnet_metadata import DOMAIN_NET_DOMAINS, DOMAIN_NET_CLASSES, DOMAIN_NET_DIVISIONS

from torch.utils.data import DataLoader
from torchvision.transforms import v2 as T

from yasin_utils.image import imagenet_normalize

In [2]:
division_map = {}
for k, v in DOMAIN_NET_DIVISIONS.items():
    for i in v:
        division_map[i] = k

In [3]:
# root = '/data/domainnet_v1.0'
# set_map = []
# for domain in DOMAIN_NET_DOMAINS:
#     try:
#         labels = os.listdir(os.path.join(root, domain))
#     except:
#         raise Exception(f'{domain} directory not found.')
#     for label in labels:
#         for image in os.listdir(os.path.join(root, domain, label)):
#             if label in division_map:
#                 set_map.append(
#                     dict(
#                         img_path=os.path.join(root, domain, label, image),
#                         label=division_map[label],
#                         domain=domain
#                         )
#                     )

In [4]:
# random.shuffle(set_map)

In [5]:
# train_set_map, test_set_map = set_map[:int(0.9*len(set_map))], set_map[int(0.9*len(set_map)):]

In [6]:
with open('/home/yasin/repos/dispatch_smol/pretraining/data_modules/train_set_map_domainnet.json') as f:
    train_set_map_ = json.load(f)
with open('/home/yasin/repos/dispatch_smol/pretraining/data_modules/test_set_map_domainnet.json') as f:
    test_set_map_ = json.load(f)

In [7]:
train_set_map = []
for sample in train_set_map_:
    if sample['label'] in division_map:
        new_sample = {**sample}
        new_sample['label'] = division_map[sample['label']]
        train_set_map.append(new_sample)

In [8]:
test_set_map = []
for sample in test_set_map_:
    if sample['label'] in division_map:
        new_sample = {**sample}
        new_sample['label'] = division_map[sample['label']]
        test_set_map.append(new_sample)

In [9]:
len(train_set_map), len(test_set_map)

(241485, 26659)

In [10]:
dict(collections.Counter([(d['label'], d['domain']) for d in train_set_map]))

{('tool', 'quickdraw'): 12600,
 ('tool', 'painting'): 4618,
 ('electricity', 'real'): 10563,
 ('cloth', 'infograph'): 3370,
 ('tool', 'real'): 11645,
 ('mammal', 'quickdraw'): 11309,
 ('cloth', 'real'): 10397,
 ('tool', 'clipart'): 3441,
 ('mammal', 'sketch'): 4644,
 ('electricity', 'painting'): 2669,
 ('furniture', 'painting'): 4472,
 ('cloth', 'painting'): 3192,
 ('furniture', 'sketch'): 6760,
 ('building', 'quickdraw'): 9479,
 ('furniture', 'real'): 15417,
 ('building', 'real'): 10874,
 ('mammal', 'clipart'): 3096,
 ('furniture', 'quickdraw'): 15745,
 ('mammal', 'infograph'): 3212,
 ('mammal', 'real'): 13996,
 ('electricity', 'infograph'): 3744,
 ('electricity', 'quickdraw'): 11284,
 ('cloth', 'sketch'): 4374,
 ('building', 'painting'): 4610,
 ('mammal', 'painting'): 8114,
 ('electricity', 'clipart'): 2803,
 ('cloth', 'clipart'): 3275,
 ('building', 'sketch'): 4241,
 ('furniture', 'clipart'): 5199,
 ('tool', 'sketch'): 4373,
 ('cloth', 'quickdraw'): 10285,
 ('building', 'clipart'): 

In [11]:
# balance_map = {
#     'furniture': {'main_domain': 'clipart', 'target': 5_253},
#     'cloth': {'main_domain': 'real', 'target': 10_377},
#     'electricity': {'main_domain': 'infograph', 'target': 3_745},
#     'building': {'main_domain': 'painting', 'target': 4_570},
#     'mammal': {'main_domain': 'quickdraw', 'target': 11_202},
#     'tool': {'main_domain': 'sketch', 'target': 4_401},
# }

In [12]:
balance_map = {
    'furniture': {'main_domain': 'clipart', 'target': 100},
    'cloth': {'main_domain': 'real', 'target': 100},
    'electricity': {'main_domain': 'infograph', 'target': 100},
    'building': {'main_domain': 'painting', 'target': 100},
    'mammal': {'main_domain': 'quickdraw', 'target': 100},
    'tool': {'main_domain': 'sketch', 'target': 100},
}

In [13]:
# balance_map = {
#     'furniture': {'main_domain': 'clipart', 'target': 1_000},
#     'cloth': {'main_domain': 'real', 'target': 1_000},
#     'electricity': {'main_domain': 'infograph', 'target': 1_000},
#     'building': {'main_domain': 'painting', 'target': 1_000},
#     'mammal': {'main_domain': 'quickdraw', 'target': 1_000},
#     'tool': {'main_domain': 'sketch', 'target': 1_000},
# }

In [14]:
# balance_map = {
#     'furniture': {'main_domain': 'clipart', 'target': 100},
#     'cloth': {'main_domain': 'real', 'target': 100},
#     'electricity': {'main_domain': 'infograph', 'target': 100},
#     'building': {'main_domain': 'painting', 'target': 100},
#     'mammal': {'main_domain': 'quickdraw', 'target': 100},
#     'tool': {'main_domain': 'sketch', 'target': 100},
# }

In [15]:
def prune(set_map, balance_map):
    pruned_set_map = []
    
    domains = set([i['domain'] for i in set_map])
    counts = {
        l: {
            d: 0 for d in domains
        } for l in balance_map
    }
    targets = {
        l: {
            d: balance_map[l]['target'] if d == balance_map[l]['main_domain'] else int(balance_map[l]['target']*0.02) for d in domains
        } for l in balance_map
    }

    for i in set_map:
        label, domain = i['label'], i['domain']

        if counts[label][domain] < targets[label][domain]:
            pruned_set_map.append(i)

        counts[label][domain] += 1

    return pruned_set_map

In [16]:
train_set_map = prune(train_set_map, balance_map)

In [17]:
dict(collections.Counter([(d['label'], d['domain']) for d in train_set_map]))

{('tool', 'quickdraw'): 2,
 ('tool', 'painting'): 2,
 ('electricity', 'real'): 2,
 ('cloth', 'infograph'): 2,
 ('tool', 'real'): 2,
 ('mammal', 'quickdraw'): 100,
 ('cloth', 'real'): 100,
 ('tool', 'clipart'): 2,
 ('mammal', 'sketch'): 2,
 ('electricity', 'painting'): 2,
 ('furniture', 'painting'): 2,
 ('cloth', 'painting'): 2,
 ('furniture', 'sketch'): 2,
 ('building', 'quickdraw'): 2,
 ('furniture', 'real'): 2,
 ('building', 'real'): 2,
 ('mammal', 'clipart'): 2,
 ('furniture', 'quickdraw'): 2,
 ('mammal', 'infograph'): 2,
 ('mammal', 'real'): 2,
 ('electricity', 'infograph'): 100,
 ('electricity', 'quickdraw'): 2,
 ('cloth', 'sketch'): 2,
 ('building', 'painting'): 100,
 ('mammal', 'painting'): 2,
 ('electricity', 'clipart'): 2,
 ('cloth', 'clipart'): 2,
 ('building', 'sketch'): 2,
 ('furniture', 'clipart'): 100,
 ('tool', 'sketch'): 100,
 ('cloth', 'quickdraw'): 2,
 ('building', 'clipart'): 2,
 ('building', 'infograph'): 2,
 ('furniture', 'infograph'): 2,
 ('electricity', 'sketch')

In [18]:
class UnbalancedDomainNetDataset(ImageDataset):
    def __init__(self, set_map, transform=None) -> None:

        super().__init__(set_map, transform)
        self.class_map = {
            'furniture': 0,
            'cloth': 1,
            'electricity': 2,
            'building': 3,
            'mammal': 4,
            'tool': 5,
        }
        self.domain_map = {
            'clipart': 0,
            'real': 1,
            'infograph': 2,
            'painting': 3,
            'quickdraw': 4,
            'sketch': 5,
        }

    def __getitem__(self, index):
        item = super().__getitem__(index)
        item['label'] = self.class_map[item['label']]
        item['domain'] = self.domain_map[item['domain']]

        return item

In [19]:
train_transform = T.Compose([
    # T.RandomResizedCrop(128),
    # T.RandomHorizontalFlip(),
    T.Resize((128,128)),
    T.ToTensor(),
    imagenet_normalize,
])
val_transform = T.Compose([
    # T.Resize(156),
    # T.CenterCrop(128),
    T.Resize((128,128)),
    T.ToTensor(),
    imagenet_normalize
])

/home/yasin/miniforge3/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [20]:
train_set = UnbalancedDomainNetDataset(set_map=train_set_map, transform=train_transform)
test_set = UnbalancedDomainNetDataset(set_map=test_set_map, transform=val_transform)

In [21]:
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=8, pin_memory=True, persistent_workers=True, drop_last=True)
val_loader = DataLoader(test_set, batch_size=256, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)

In [22]:
len(train_set), len(test_set)

(660, 26659)

## Train Models

In [23]:
from typing import OrderedDict
import torch
from torch import nn
from torchvision.models.resnet import resnet50
import pytorch_lightning as L
from torchmetrics import Accuracy

In [24]:
class LinearProbe(L.LightningModule):
    def __init__(self, backbone, emb_dim, num_classes, lr=1e-3) -> None:
        super().__init__()
        
        self.backbone: nn.Module = backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        self.backbone.eval()

        self.linear_head: nn.Module = nn.Linear(emb_dim, num_classes)
        # self.linear_head: nn.Module = nn.Sequential(
        #     nn.Linear(emb_dim, 1024),
        #     nn.ReLU(),
        #     nn.Linear(1024, 512),
        #     nn.Linear(512, num_classes)
        # ) 
        
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr

        self.test_accuracy = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.linear_head(x)

        return x

    def training_step(self, batch, bacth_idx):
        X = batch['image']
        t = batch['label']

        y = self.forward(X)
        loss = self.criterion(y, t)

        print(loss.item())

        return loss
    
    def test_step(self, batch, batch_idx):
        X = batch['image']
        t = batch['label']

        y = self.forward(X)
        acc = self.test_accuracy(y, t)

        self.log('accuracy', acc, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.linear_head.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 20)

        return [optimizer], [scheduler]

In [25]:
def get_backbone_from_ckpt(ckpt_path: str) -> torch.nn.Module:
    state_dict = torch.load(ckpt_path)["state_dict"]
    state_dict = OrderedDict([
        (".".join(name.split(".")[1:]), param) for name, param in state_dict.items() if name.startswith("backbone")
    ])

    return state_dict

In [26]:
# Baseline Model
model_bl  = resnet50()
weights_bl = get_backbone_from_ckpt("/home/yasin/Downloads/final_baseline.ckpt")
model_bl.load_state_dict(weights_bl, strict=False)
model_bl.fc = torch.nn.Identity()
model_bl = model_bl.cuda()

In [27]:
# MixStyle Model
model_ms  = resnet50()
weights_ms = get_backbone_from_ckpt("/home/yasin/Downloads/final_mixstyle.ckpt")
model_ms.load_state_dict(weights_ms, strict=False)
model_ms.fc = torch.nn.Identity()
model_ms = model_ms.cuda()

In [28]:
baseline_module = LinearProbe(model_bl, emb_dim=2048, num_classes=6, lr=1e-3)

In [29]:
mixstyle_module = LinearProbe(model_ms, emb_dim=2048, num_classes=6, lr=1e-3)

In [30]:
trainer = L.Trainer(
    max_epochs=20
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [31]:
trainer.fit(baseline_module, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | backbone      | ResNet             | 23.5 M
1 | linear_head   | Linear             | 12.3 K
2 | criterion     | CrossEntropyLoss   | 0     
3 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
12.3 K    Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.081    Total estimated model params size (MB)
/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

1.848394751548767
1.6978671550750732
1.5292961597442627
1.4333938360214233
1.2815630435943604
1.1749796867370605
1.0765836238861084
0.9487751722335815
0.8977640867233276
0.8391698598861694
0.7664602994918823
0.738565981388092
0.6487065553665161
0.6659150719642639
0.6047763228416443
0.6039161682128906
0.502172589302063
0.5991513133049011
0.575089693069458
0.48921364545822144
0.53260737657547
0.48354169726371765
0.474668025970459
0.46403902769088745
0.5051780939102173
0.4321802258491516
0.4562263488769531
0.44223126769065857
0.4226888120174408
0.4479796588420868
0.4155516028404236
0.4232303500175476
0.45911839604377747
0.42386698722839355
0.41045141220092773
0.4452684819698334
0.41823261976242065
0.4454236328601837
0.36650848388671875
0.4826802611351013


`Trainer.fit` stopped: `max_epochs=20` reached.


In [32]:
trainer.test(baseline_module, dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 256. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │        0.88671875         │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.88671875}]

In [33]:
trainer.test(baseline_module, dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 35. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.2997486889362335     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.2997486889362335}]

In [34]:
trainer = L.Trainer(
    max_epochs=20
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [35]:
trainer.fit(mixstyle_module, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | backbone      | ResNet             | 23.5 M
1 | linear_head   | Linear             | 12.3 K
2 | criterion     | CrossEntropyLoss   | 0     
3 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
12.3 K    Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.081    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

1.8548887968063354
1.7114722728729248
1.5360091924667358
1.4342327117919922
1.304953932762146
1.1808087825775146
1.1333686113357544
0.982710599899292
0.9016388058662415
0.8918410539627075
0.8006599545478821
0.7706559896469116
0.6919435262680054
0.6780275702476501
0.6313337683677673
0.6375774145126343
0.6314430236816406
0.6137170791625977
0.5670182704925537
0.5655918717384338
0.5592472553253174
0.5304793119430542
0.5131610631942749
0.5166656970977783
0.5268531441688538
0.47509777545928955
0.42184948921203613
0.5753235816955566
0.4534452557563782
0.5093572735786438
0.4301510751247406
0.5087066292762756
0.487271249294281
0.46189698576927185
0.4839189648628235
0.475533127784729
0.447551965713501
0.46548572182655334
0.41900771856307983
0.46061593294143677


`Trainer.fit` stopped: `max_epochs=20` reached.


In [36]:
trainer.test(mixstyle_module, dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │        0.88671875         │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.88671875}]

In [37]:
trainer.test(mixstyle_module, dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.3087887763977051     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.3087887763977051}]

| # Samples | Baseline | MixStyle |
| --------- | -------- | -------- |
| All       | 0.629    | 0.642    |
| 1_000     | 0.569    | 0.576    |
| 500       | 0.520    | 0.538    |
| 100       | 0.451    | 0.463    |